In [1]:
import pandas as pd
import warnings
from unidecode import unidecode
from sportsreference.ncaab.roster import Player
from re import search
from tqdm import tqdm, tqdm_notebook
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
from sportsreference.ncaab.roster import Player

In [3]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [4]:
df = pd.read_csv("NBA_data.csv")

In [84]:
df.head(20)

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,Cincinnati,15,23134,9325,Above Avg
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,Indiana,3,1246,442,Below Avg
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,UConn,9,6443,1814,Below Avg
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,UConn,1,936,314,Bust
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,University of Louisiana at Monroe,1,180,51,Bust
5,20.0,17.3,Eddie House,717,2000,37,7.5,0.077,-1.5,1.6,MIA,Arizona State,11,12405,5356,Above Avg
6,24.7,18.1,Eduardo Nájera,619,2000,38,4.9,0.106,0.3,6.4,HOU,Oklahoma,12,11213,3052,Above Avg
7,-0.2,7.3,Lavor Postell,61,2000,39,3.2,-0.025,-8.6,-0.7,NYK,St. John's,3,446,195,Below Avg
8,0.9,15.2,Hanno Möttölä,155,2000,40,4.6,0.019,-5.8,-2.3,ATL,Utah,2,2360,715,Bust
9,0.0,0.0,Chris Carrawell,0,2000,41,0.0,0.000,0.0,0.0,SAS,Duke,0,0,0,Bust


In [5]:
#remove accents 
clean_player = [unidecode(player) for player in df["Player"]]
clean_player[:10]

['Kenyon Martin',
 'A.J. Guyton',
 'Jake Voskuhl',
 'Khalid El-Amin',
 'Mike Smith',
 'Eddie House',
 'Eduardo Najera',
 'Lavor Postell',
 'Hanno Mottola',
 'Chris Carrawell']

In [6]:
#make player ids
suffix = "-1"
makes_ids = [player.replace(".", "").replace("'", "").replace(" ", "-").lower()+suffix for player in clean_player]
makes_ids[:10]

['kenyon-martin-1',
 'aj-guyton-1',
 'jake-voskuhl-1',
 'khalid-el-amin-1',
 'mike-smith-1',
 'eddie-house-1',
 'eduardo-najera-1',
 'lavor-postell-1',
 'hanno-mottola-1',
 'chris-carrawell-1']

In [ ]:
#see which players are generating errors when trying to call college data 
for player in makes_ids:
    try:
        college = Player(player)
    except:
        print(f'Error for {player} in index # {makes_ids.index(player)}')
        


#exceptions
deeandre hullett, cc 23
kedrick brown, cc, 78
ronald murray, cc, 93
qyntel woods, cc, 128
jerome beasely, cc, 139
donta smith, cc, 172
luc-richard-mbah-a-moute-1, 337
bill-walker-1, 343
patrick-mills-1, 398
jeff-pendergraph-1, 403
bj-mullens-1, 424
bernard james, cc, 533
thomas-robinson-2, 555
ricky ledo, cc, 579
roy-devyn-marble-1, 644
kyle-anderson-3, 650
stephen-zimmermanjr-1, 720
kahlil-felder-1, 732
jakob-poeltl-1, 748
taurean-prince-1, 750
edrice-adebayo-1, 799
malik-milton-1, 830
raymond-spalding-1, 831
marvin-bagleyiii-1, 839
lonnie-walker-2, 853

In [7]:
#make list to fix incorrect player id's 
replace = [('luc-richard-mbah-a-moute-1', 337), ('bill-walker-1', 343), ('patrick-mills-1', 398), ('jeff-pendergraph-1', 403), 
          ('bj-mullens-1', 424),('thomas-robinson-2', 555), ('roy-devyn-marble-1', 644), ('kyle-anderson-3', 650), 
           ('stephen-zimmermanjr-1', 720), ('kahlil-felder-1', 732),('jakob-poeltl-1', 748), ('taurean-prince-1', 750),
           ('edrice-adebayo-1', 799), ('malik-milton-1', 830),('raymond-spalding-1', 831), ('marvin-bagleyiii-1', 839),
           ('lonnie-walker-2', 853)]

In [8]:
#replace incorrect id's
for player in replace:
    makes_ids[player[1]] = player[0]

In [9]:
#test corrected list 
makes_ids[337]

'luc-richard-mbah-a-moute-1'

In [10]:
#list players to remove
remove = ['deeandre-hulett-1', 'kedrick-brown-1', 'ronald-murray-1', 'qyntel-woods-1',
'jerome-beasley-1', 'donta-smith-1','bernard-james-1', 'ricky-ledo-1']
no_cc = [player for player in makes_ids if player not in remove] 
len(no_cc)

855

In [11]:
#list index of players to remove
remove_nums = [23, 78, 93, 128, 139, 172, 533, 579]

In [12]:
#remove same index rows from df
clean_df = df.drop(remove_nums)

In [13]:
#add Player_id column to lean NBA stats df
clean_df['Player_id'] = no_cc

In [14]:
#get colleges of players to make sure we have the correct players
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [15]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [16]:
#add to clean_df
clean_df['College_id'] = college_list

In [17]:
#fix college abbreviations
clean_df["College_id"]= (clean_df["College_id"]
.replace("connecticut", "uconn").replace("louisiana state", "lsu").replace("southern california", "usc")
.replace('north carolina', 'unc').replace('southern methodist', 'smu').replace('brigham young', 'byu')
.replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
.replace('massachusetts', 'umass').replace('pittsburgh', 'pitt').replace('louisiana lafayette', 'la-lafayette'))


In [18]:
#convert College column to lower case 
clean_df['College'] = clean_df['College'].str.lower() 

In [19]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['College_id']]
clean_df['word'] = word

In [20]:
wrong_players = []
for index, row in clean_df.iterrows():
    if search(row["word"], row["College"]):
        print(row["Player"], "at ", index, " matches")
    else:
        print(row["Player"], "at ", index, " does NOT match")
        wrong_players.append((row["Player"]))


Kenyon Martin at  0  matches
A.J. Guyton at  1  matches
Jake Voskuhl at  2  matches
Khalid El-Amin at  3  matches
Mike Smith at  4  matches
Eddie House at  5  matches
Eduardo Nájera at  6  matches
Lavor Postell at  7  matches
Hanno Möttölä at  8  matches
Chris Carrawell at  9  matches
Michael Redd at  10  matches
Dan Langhi at  11  matches
Jabari Smith at  12  matches
Mark Karcher at  13  matches
Jason Hart at  14  matches
Kaniel Dickens at  15  matches
Ernest Brown at  16  does NOT match
Dan McClintock at  17  matches
Cory Hightower at  18  does NOT match
Chris Porter at  19  matches
Jaquay Walls at  20  matches
Scoonie Penn at  21  matches
Pete Mickeal at  22  matches
Brian Cardinal at  24  matches
Erick Barkley at  25  matches
Mark Madsen at  26  matches
Stromile Swift at  27  matches
Marcus Fizer at  28  matches
Mike Miller at  29  matches
DerMarr Johnson at  30  matches
Chris Mihm at  31  matches
Jamal Crawford at  32  matches
Keyon Dooling at  33  matches
Jérôme Moïso at  34  mat

Maurice Ager at  275  matches
Jordan Farmar at  276  matches
Kyle Lowry at  277  matches
Josh Boone at  278  matches
Shannon Brown at  279  matches
Rajon Rondo at  280  matches
Renaldo Balkman at  281  matches
Quincy Douby at  282  matches
Shawne Williams at  283  matches
Marcus Williams at  284  matches
Adam Haluska at  285  matches
Derrick Byars at  286  matches
Chris Richard at  287  matches
Marcus Williams at  288  does NOT match
Josh McRoberts at  289  matches
Jermareo Davidson at  290  matches
Glen Davis at  291  matches
Nick Fazekas at  292  matches
Reyshawn Terry at  293  matches
Jared Jordan at  294  matches
Taurean Green at  295  matches
Dominic McGuire at  296  matches
Aaron Gray at  297  matches
JamesOn Curry at  298  matches
Demetris Nichols at  299  matches
Herbert Hill at  300  matches
Ramon Sessions at  301  matches
Sammy Mejia at  302  matches
D.J. Strawberry at  303  matches
Gabe Pruitt at  304  matches
Stéphane Lasme at  305  matches
Carl Landry at  306  matches
Alan

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  matches
Isaiah Thomas at  503  matches
Cory Joseph at  504  matches
Kyrie Irving at  505  matches
Derrick Williams at  506  does NOT match
Tristan Thompson at  507  does NOT match
Brandon Knight at  508  matches
Kemba Walker at  509  matches
Jimmer Fredette at  510  matches
Klay Thompson at  511  matches
Alec Burks at  512  matches
Markieff Morris at  513  matches
Marcus Morris at  514  matches
Jimmy Butler at  515  matches
Nikola Vučević at  516  matches
Kawhi Leonard at  517  matches
Norris Cole at  518  matches
JaJuan Johnson at  519  matches
Jordan Hamilton at  520  does NOT match
Reggie Jackson at  521  matches
MarShon Brooks at  522  matches
Nolan Smith at  523  matches
Tobias Harris at  524  matches
Chris Singleton at  525  matches
Iman Shumpert at  526  matches
Kenneth Faried at  527  matches
Will Barton at  528  matches
Mike Scott at  529  does NOT match
Doron Lamb at  530  matches
Tyshawn Taylor at  531  matches
Khris Middleton at  532  matches
Quincy Acy at  534  matches
O

Tyler Dorsey at  762  matches
Damyean Dotson at  763  does NOT match
Thomas Bryant at  764  matches
Dwayne Bacon at  765  matches
Wesley Iwundu at  766  matches
Jordan Bell at  767  matches
Semi Ojeleye at  768  matches
Ivan Rabb at  769  matches
Frank Mason at  770  matches
Jawun Evans at  771  matches
Sterling Brown at  772  does NOT match
Sindarius Thornwell at  773  matches
Monte Morris at  774  matches
Edmond Sumner at  775  matches
Kadeem Allen at  776  matches
Alec Peters at  777  matches
Nigel Williams-Goss at  778  matches
Jabari Bird at  779  matches
Jaron Blossomgame at  780  matches
Davon Reed at  781  matches
Ike Anigbogu at  782  matches
Frank Jackson at  783  does NOT match
Dillon Brooks at  784  matches
Derrick White at  785  matches
Markelle Fultz at  786  matches
Lonzo Ball at  787  matches
Jayson Tatum at  788  matches
Josh Jackson at  789  does NOT match
De'Aaron Fox at  790  matches
Jonathan Isaac at  791  matches
Lauri Markkanen at  792  matches
Josh Hart at  793 

In [21]:
len(wrong_players)

54

In [22]:
select = clean_df["Player"].isin(wrong_players)
wrong_df = clean_df[select]

In [24]:
#create list of players to remove
remove2 = ['ernest-brown-1', 'cory-hightower-1', ]

In [25]:
#create list of player ids to update
replace = [('ken-johnson-1', 'ken-johnson-2'), ('mike-dunleavy-1', 'mike-dunleavy-2'), ('david-young-1','dave-young-1'),
          ('bobby-jones-1', 'bobby-jones-2'), ('dee-brown-1', 'dee-brown-2'), ('gerald-henderson-1', 'gerald-henderson-2'),
          ('james-johnson-1', 'james-johnson-2'), ('wesley-johnson-1', 'wes-johnson-1'),
          ('james-anderson-1', 'james-anderson-3'), ('derrick-williams-1', 'derrick-williams-2'),
          ('tristan-thompson-1','tristan-thompson-2'), ('jordan-hamilton-1', 'jordan-hamilton-2'),
          ('mike-scott-1', 'mike-scott-4'), ('justin-hamilton-1', 'justin-hamilton-2'), ('anthony-davis-1', 'anthony-davis-5'),
          ('jared-cunningham-1', 'jared-cunningham-2'), ('ray-mccallum-1', 'ray-mccallum-2'), ('glen-rice-1', 'glen-rice-jr-1'),
          ('ryan-kelly-1', 'ryan-kelly-3'), ('reggie-bullock-1', 'reggie-bullock-2'), ('tim-hardaway-1', 'tim-hardaway-jr-1'),
          ('glenn-robinson-1', 'glenn-robinson-2'),('joe-harris-1', 'joe-harris--1'),
          ('marcus-thornton-1', 'marcus-thornton-3'), ('joe-young-1', 'joseph-young-1'),
          ('josh-richardson-1', 'josh-richardson-2'), ('anthony-brown-1', 'anthony-brown-5'), ('aaron-white-1','aaron-white-3'),
          ('devin-booker-1', 'devin-booker-2'), ('larry-nance-1', 'larry-nance-2'),
          ('demetrius-jackson-1', 'demetrius-jackson-2'), ('sterling-brown-1', 'sterling-brown-2'),
          ('frank-jackson-1', 'frank-jackson-2'), ('josh-jackson-1', 'josh-jackson-2'), ('dennis-smith-1', 'dennis-smithjr-1'),
          ('justin-jackson-1', 'justin-jackson-4'), ('justin-jackson-1', 'justin-jackson-5'),
          ('bruce-brown-1', 'bruce-brown-2'), ('gary-trent-1', 'gary-trentjr-1'), ('vince-edwards-1', 'vince-edwards-2'),
          ('troy-brown-1', 'troy-brown-5'), ('jaren-jackson-1', 'jaren-jacksonjr-1'), 
          ('wendell-carter-1', 'wendell-carterjr-1'),('jerome-robinson-1', 'jerome-robinson-2'),
          ('marcus-williams-1', 'marcus-williams-2'), ('michael-porter-1', 'michael-porterjr-1'),
          ('robert-williams-1', 'robert-williams-3')
          ]

In [26]:
#update player ids in clean_df
for player in replace:
    clean_df.replace(to_replace = player[0],
                    value = player[1], inplace = True)

In [27]:
#input justin jackson 5
clean_df.loc[812, 'Player_id'] = 'justin-jackson-5'

In [28]:
#remove players not in NCAA
clean_df = clean_df[~clean_df['Player_id'].isin(remove2)]

In [29]:
#test player ids one more time
college_list = []
for player in tqdm_notebook(clean_df["Player_id"]):
    try:
        college = Player(player).team_abbreviation
        college_list.append(college)
    except:
        print(f'Error for {player}')
        

In [30]:
#replace hyphenated shcool names
college_list = [school.replace("-", " ") for school in college_list]

In [31]:
#add to clean_df
clean_df['College_id'] = college_list

In [32]:
#fix college abbreviations
clean_df["College_id"]= (clean_df["College_id"]
.replace("connecticut", "uconn").replace("louisiana state", "lsu").replace("southern california", "usc")
.replace('north carolina', 'unc').replace('southern methodist', 'smu').replace('brigham young', 'byu')
.replace('nevada las vegas', 'unlv').replace('mississippi', 'ole miss').replace('north carolina state', 'nc state')
.replace('massachusetts', 'umass').replace('pittsburgh', 'pitt').replace('louisiana lafayette', 'la-lafayette'))

In [33]:
#create word column used for college matching
word = [words.split()[0] for words in clean_df['College_id']]
clean_df['word'] = word

In [34]:
#test player ids one more time
wrong_players = []
for index, row in clean_df.iterrows():
    if search(row["word"], row["College"]):
        print(row["Player"], "at ", index, " matches")
    else:
        print(row["Player"], "at ", index, " does NOT match")
        wrong_players.append((row["Player"], index))

Kenyon Martin at  0  matches
A.J. Guyton at  1  matches
Jake Voskuhl at  2  matches
Khalid El-Amin at  3  matches
Mike Smith at  4  matches
Eddie House at  5  matches
Eduardo Nájera at  6  matches
Lavor Postell at  7  matches
Hanno Möttölä at  8  matches
Chris Carrawell at  9  matches
Michael Redd at  10  matches
Dan Langhi at  11  matches
Jabari Smith at  12  matches
Mark Karcher at  13  matches
Jason Hart at  14  matches
Kaniel Dickens at  15  matches
Dan McClintock at  17  matches
Chris Porter at  19  matches
Jaquay Walls at  20  matches
Scoonie Penn at  21  matches
Pete Mickeal at  22  matches
Brian Cardinal at  24  matches
Erick Barkley at  25  matches
Mark Madsen at  26  matches
Stromile Swift at  27  matches
Marcus Fizer at  28  matches
Mike Miller at  29  matches
DerMarr Johnson at  30  matches
Chris Mihm at  31  matches
Jamal Crawford at  32  matches
Keyon Dooling at  33  matches
Jérôme Moïso at  34  matches
Etan Thomas at  35  matches
Courtney Alexander at  36  matches
Mateen

Shannon Brown at  279  matches
Rajon Rondo at  280  matches
Renaldo Balkman at  281  matches
Quincy Douby at  282  matches
Shawne Williams at  283  matches
Marcus Williams at  284  does NOT match
Adam Haluska at  285  matches
Derrick Byars at  286  matches
Chris Richard at  287  matches
Marcus Williams at  288  matches
Josh McRoberts at  289  matches
Jermareo Davidson at  290  matches
Glen Davis at  291  matches
Nick Fazekas at  292  matches
Reyshawn Terry at  293  matches
Jared Jordan at  294  matches
Taurean Green at  295  matches
Dominic McGuire at  296  matches
Aaron Gray at  297  matches
JamesOn Curry at  298  matches
Demetris Nichols at  299  matches
Herbert Hill at  300  matches
Ramon Sessions at  301  matches
Sammy Mejia at  302  matches
D.J. Strawberry at  303  matches
Gabe Pruitt at  304  matches
Stéphane Lasme at  305  matches
Carl Landry at  306  matches
Alando Tucker at  307  matches
Greg Oden at  308  matches
Kevin Durant at  309  matches
Al Horford at  310  matches
Mike 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Alec Burks at  512  matches
Markieff Morris at  513  matches
Marcus Morris at  514  matches
Jimmy Butler at  515  matches
Nikola Vučević at  516  matches
Kawhi Leonard at  517  matches
Norris Cole at  518  matches
JaJuan Johnson at  519  matches
Jordan Hamilton at  520  matches
Reggie Jackson at  521  matches
MarShon Brooks at  522  matches
Nolan Smith at  523  matches
Tobias Harris at  524  matches
Chris Singleton at  525  matches
Iman Shumpert at  526  matches
Kenneth Faried at  527  matches
Will Barton at  528  matches
Mike Scott at  529  matches
Doron Lamb at  530  matches
Tyshawn Taylor at  531  matches
Khris Middleton at  532  matches
Quincy Acy at  534  matches
Orlando Johnson at  535  does NOT match
Draymond Green at  536  matches
Jae Crowder at  537  matches
Kim English at  538  matches
Quincy Miller at  539  matches
Justin Hamilton at  540  matches
Kevin Murphy at  541  matches
Kyle O'Quinn at  542  matches
Kris Joseph at  543  matches
Darius Johnson-Odom at  544  matches
Rob

Kadeem Allen at  776  matches
Alec Peters at  777  matches
Nigel Williams-Goss at  778  matches
Jabari Bird at  779  matches
Jaron Blossomgame at  780  matches
Davon Reed at  781  matches
Ike Anigbogu at  782  matches
Frank Jackson at  783  matches
Dillon Brooks at  784  matches
Derrick White at  785  matches
Markelle Fultz at  786  matches
Lonzo Ball at  787  matches
Jayson Tatum at  788  matches
Josh Jackson at  789  matches
De'Aaron Fox at  790  matches
Jonathan Isaac at  791  matches
Lauri Markkanen at  792  matches
Josh Hart at  793  matches
Dennis Smith at  794  matches
Zach Collins at  795  matches
Malik Monk at  796  matches
Luke Kennard at  797  matches
Donovan Mitchell at  798  matches
Bam Adebayo at  799  matches
Justin Patton at  800  matches
Justin Jackson at  801  matches
Tony Bradley at  802  matches
Kyle Kuzma at  803  matches
Caleb Swanigan at  804  matches
Tyler Lydon at  805  matches
OG Anunoby at  806  matches
Harry Giles at  807  matches
John Collins at  808  match

In [35]:
len(wrong_players)

8

In [36]:
#all 'wrong players' are fine in this case.
wrong_players

[('David Young', 167),
 ('Robert Whaley', 211),
 ('Marcus Williams', 284),
 ('Trent Plaisted', 354),
 ('Marcus Thornton', 382),
 ('Orlando Johnson', 535),
 ('Damyean Dotson', 763),
 ('Kevin Hervey', 832)]

In [134]:
clean_df.head(20)

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class,Player_id,College_id,word
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,cincinnati,15,23134,9325,Above Avg,kenyon-martin-1,cincinnati,cincinnati
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,indiana,3,1246,442,Below Avg,aj-guyton-1,indiana,indiana
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,uconn,9,6443,1814,Below Avg,jake-voskuhl-1,uconn,uconn
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,uconn,1,936,314,Bust,khalid-el-amin-1,uconn,uconn
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,university of louisiana at monroe,1,180,51,Bust,mike-smith-1,louisiana monroe,louisiana
5,20.0,17.3,Eddie House,717,2000,37,7.5,0.077,-1.5,1.6,MIA,arizona state,11,12405,5356,Above Avg,eddie-house-1,arizona state,arizona
6,24.7,18.1,Eduardo Nájera,619,2000,38,4.9,0.106,0.3,6.4,HOU,oklahoma,12,11213,3052,Above Avg,eduardo-najera-1,oklahoma,oklahoma
7,-0.2,7.3,Lavor Postell,61,2000,39,3.2,-0.025,-8.6,-0.7,NYK,st. john's,3,446,195,Below Avg,lavor-postell-1,st johns ny,st
8,0.9,15.2,Hanno Möttölä,155,2000,40,4.6,0.019,-5.8,-2.3,ATL,utah,2,2360,715,Bust,hanno-mottola-1,utah,utah
9,0.0,0.0,Chris Carrawell,0,2000,41,0.0,0.000,0.0,0.0,SAS,duke,0,0,0,Bust,chris-carrawell-1,duke,duke


In [135]:
#re-index clean_df
clean_df.reset_index(drop=True, inplace=True)

In [137]:
clean_df.head(20)

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class,Player_id,College_id,word
0,48.0,30.6,Kenyon Martin,757,2000,1,12.3,0.100,0.7,15.5,NJN,cincinnati,15,23134,9325,Above Avg,kenyon-martin-1,cincinnati,cincinnati
1,0.3,15.6,A.J. Guyton,80,2000,32,5.5,0.010,-5.4,-1.1,CHI,indiana,3,1246,442,Below Avg,aj-guyton-1,indiana,indiana
2,11.6,14.3,Jake Voskuhl,450,2000,33,4.0,0.087,-2.2,-0.3,CHI,uconn,9,6443,1814,Below Avg,jake-voskuhl-1,uconn,uconn
3,0.6,18.7,Khalid El-Amin,50,2000,34,6.3,0.030,-3.8,-0.4,CHI,uconn,1,936,314,Bust,khalid-el-amin-1,uconn,uconn
4,-0.1,10.6,Mike Smith,17,2000,35,3.0,-0.033,-5.5,-0.2,WAS,university of louisiana at monroe,1,180,51,Bust,mike-smith-1,louisiana monroe,louisiana
5,20.0,17.3,Eddie House,717,2000,37,7.5,0.077,-1.5,1.6,MIA,arizona state,11,12405,5356,Above Avg,eddie-house-1,arizona state,arizona
6,24.7,18.1,Eduardo Nájera,619,2000,38,4.9,0.106,0.3,6.4,HOU,oklahoma,12,11213,3052,Above Avg,eduardo-najera-1,oklahoma,oklahoma
7,-0.2,7.3,Lavor Postell,61,2000,39,3.2,-0.025,-8.6,-0.7,NYK,st. john's,3,446,195,Below Avg,lavor-postell-1,st johns ny,st
8,0.9,15.2,Hanno Möttölä,155,2000,40,4.6,0.019,-5.8,-2.3,ATL,utah,2,2360,715,Bust,hanno-mottola-1,utah,utah
9,0.0,0.0,Chris Carrawell,0,2000,41,0.0,0.000,0.0,0.0,SAS,duke,0,0,0,Bust,chris-carrawell-1,duke,duke


In [ ]:
# d = []
# for p in game.players.passing():
#     d.append({'Player': p, 'Team': p.team, 'Passer Rating':
#         p.passer_rating()})

# pd.DataFrame(d)

In [138]:
#create  list to hold dictionary data and list for desired ncaa stats
ncaa_data = []

stats = ['player_id','name','assist_percentage', 'assists', 'block_percentage','blocks','effective_field_goal_percentage',
        'field_goal_percentage', 'free_throw_attempt_rate', 'free_throw_percentage', 'minutes_played',
        'points', 'steal_percentage', 'steals', 'three_point_percentage', 'three_pointers', 'total_rebound_percentage',
         'total_rebounds', 'true_shooting_percentage', 'turnover_percentage','turnovers', 'usage_percentage', 'box_plus_minus', 'games_played', 'height', 'win_shares_per_40_minutes']



In [101]:
len(stats)

28

In [139]:
#insert player stats into ncaa_data
ncaa_data = []
for player in tqdm_notebook(clean_df["Player_id"]):
    ncaa = Player(player)
    player_list = []
    player_dict = {}
    for stat in stats:
        try:
            var = eval(f'ncaa.{stat}')
            player_list.append((stat, var))
        except:
            var = np.nan
            print(player,stat, var)
            player_list.append((stat, var))
    player_dict.update(player_list)
    ncaa_data.append(player_dict)
    

In [159]:
ncaa_data[9]

{'player_id': 'chris-carrawell-1',
 'name': 'Chris Carrawell',
 'assist_percentage': None,
 'assists': 309,
 'block_percentage': None,
 'blocks': 114,
 'effective_field_goal_percentage': 0.514,
 'field_goal_percentage': 0.486,
 'free_throw_attempt_rate': 0.415,
 'free_throw_percentage': 0.667,
 'minutes_played': None,
 'points': 1455,
 'steal_percentage': None,
 'steals': 105,
 'three_point_percentage': 0.36,
 'three_pointers': 62,
 'total_rebound_percentage': None,
 'total_rebounds': 608,
 'true_shooting_percentage': 0.545,
 'turnover_percentage': None,
 'turnovers': None,
 'usage_percentage': None,
 'box_plus_minus': None,
 'games_played': 136,
 'height': '6-6',
 'win_shares_per_40_minutes': None}

In [141]:
#convert ncaa_data to df
ncaa_df = pd.DataFrame(ncaa_data)


In [120]:
len(ncaa_df)

853

In [121]:
len(clean_df)

853

In [142]:
ncaa_df.iloc[[16]]

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,name,player_id,points,steal_percentage,steals,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,usage_percentage,win_shares_per_40_minutes
16,NaN,79,NaN,196,NaN,0.632,0.632,0.499,0.652,115,7-0,1716.0,Dan McClintock,dan-mcclintock-1,1363,NaN,40,0.0,0.0,NaN,486,0.642,15.6,155.0,NaN,0.256


In [143]:
#add college to ncaa_df
ncaa_df["college"] = clean_df["College_id"]
ncaa_df["class"] = clean_df["Class"]

In [144]:
ncaa_df["class"][16]

'Bust'

In [145]:
ncaa_df.iloc[[16]]

,assist_percentage,assists,block_percentage,blocks,box_plus_minus,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,name,player_id,points,steal_percentage,steals,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,usage_percentage,win_shares_per_40_minutes,college,class
16,NaN,79,NaN,196,NaN,0.632,0.632,0.499,0.652,115,7-0,1716.0,Dan McClintock,dan-mcclintock-1,1363,NaN,40,0.0,0.0,NaN,486,0.642,15.6,155.0,NaN,0.256,northern arizona,Bust


In [146]:
#create new dataframe which re-orders columns and drops those with a lot of NaN's
ncaa2_df = ncaa_df[["player_id", "name",  "class", "college", "assists", "blocks", "effective_field_goal_percentage",
                    "field_goal_percentage", "free_throw_attempt_rate","free_throw_percentage", "games_played", "height",
                    "minutes_played", "points","steals", "three_point_percentage", "three_pointers", "total_rebounds",
                    "true_shooting_percentage", "turnover_percentage", "turnovers", "win_shares_per_40_minutes" ]]

In [147]:
ncaa2_df.iloc[[16]]

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes
16,dan-mcclintock-1,Dan McClintock,Bust,northern arizona,79,196,0.632,0.632,0.499,0.652,115,7-0,1716.0,1363,40,0.0,0.0,486,0.642,15.6,155.0,0.256


In [169]:
ncaa2_df.fillna(ncaa2_df.mean(), inplace=True)

In [161]:
#stats per 40  list
per_40 = [ "assists", "blocks", "points", "steals", "three_pointers", "total_rebounds", "turnovers"]

In [162]:
#add suffix for column titles
suff = "_per_40"
for_40 = [stat+suff for stat in per_40]

In [163]:
#zip two lists together
combined_40 = list(zip(per_40,for_40))

In [164]:
#check combined list
combined_40

[('assists', 'assists_per_40'),
 ('blocks', 'blocks_per_40'),
 ('points', 'points_per_40'),
 ('steals', 'steals_per_40'),
 ('three_pointers', 'three_pointers_per_40'),
 ('total_rebounds', 'total_rebounds_per_40'),
 ('turnovers', 'turnovers_per_40')]

In [165]:
#run for loop to create per_40 columns
for stat in combined_40:
    ncaa2_df[stat[1]] = (ncaa2_df[stat[0]] /ncaa2_df["minutes_played"])*40

In [152]:
#fill nan's with column mean
ncaa2_df.fillna(ncaa2_df.mean())

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Above Avg,cincinnati,142,292,0.587,0.586,0.494,0.581,116,6-9,2042.000000,1279,124,0.222000,2.0,869,0.592,14.300000,139.00000,0.346000,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Below Avg,indiana,403,52,0.541,0.455,0.249,0.790,128,6-1,3260.000000,2100,129,0.414000,283.0,429,0.572,13.600000,218.00000,0.145000,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Below Avg,uconn,124,193,0.542,0.542,0.555,0.656,138,6-11,2184.000000,863,86,0.329163,0.0,880,0.573,23.000000,160.00000,0.237000,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,479,10,0.486,0.416,0.257,0.822,108,5-10,2149.000000,1650,186,0.355000,195.0,319,0.527,15.600000,188.00000,0.158000,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,louisiana monroe,147,42,0.507,0.428,0.357,0.769,55,6-8,1938.000000,1062,90,0.318000,130.0,449,0.551,20.000000,241.00000,0.165000,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200
5,eddie-house-1,Eddie House,Above Avg,arizona state,405,19,0.495,0.425,0.187,0.798,124,6-1,4136.000000,2044,258,0.366000,250.0,502,0.523,10.800000,195.00000,0.113000,3.916828,0.183752,19.767892,2.495164,2.417795,4.854932,1.885880
6,eduardo-najera-1,Eduardo Najera,Above Avg,oklahoma,215,89,0.460,0.430,0.354,0.669,126,6-8,3001.000000,1646,193,0.288000,85.0,910,0.495,13.100000,201.00000,0.183000,2.865711,1.186271,21.939354,2.572476,1.132956,12.129290,2.679107
7,lavor-postell-1,Lavor Postell,Below Avg,st johns ny,205,58,0.470,0.430,0.383,0.789,129,6-5,2845.000000,1427,129,0.326000,92.0,716,0.526,13.800000,172.00000,0.115000,2.882250,0.815466,20.063269,1.813708,1.293497,10.066784,2.418278
8,hanno-mottola-1,Hanno Mottola,Bust,utah,139,50,0.533,0.500,0.415,0.774,120,6-11,2162.000000,1492,37,0.333000,71.0,551,0.580,14.900000,159.00000,0.207000,2.571693,0.925069,27.604070,0.684551,1.313599,10.194265,2.941721
9,chris-carrawell-1,Chris Carrawell,Bust,duke,309,114,0.514,0.486,0.415,0.667,136,6-6,2497.458432,1455,105,0.360000,62.0,608,0.545,15.209338,177.27896,0.181716,3.018494,1.357832,18.738180,1.505259,1.288549,8.322600,2.901975


In [184]:
#convert height to inches

height = [ guy.split('-') for guy in ncaa2_df["height"]]
inches = [(int(dim[0])*12) + int(dim[1]) for dim in height]

In [185]:
inches[:10]

[81, 73, 83, 70, 80, 73, 80, 77, 83, 78]

In [186]:
ncaa2_df['height'] = inches

In [187]:
#export to csv
ncaa2_df.to_csv("NCAA_data.csv",index = False, header = True)

In [189]:
ncaa2_df.head()

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Above Avg,cincinnati,142,292,0.587,0.586,0.494,0.581,116,81,2042.0,1279,124,0.222000,2.0,869,0.592,14.3,139.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Below Avg,indiana,403,52,0.541,0.455,0.249,0.790,128,73,3260.0,2100,129,0.414000,283.0,429,0.572,13.6,218.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Below Avg,uconn,124,193,0.542,0.542,0.555,0.656,138,83,2184.0,863,86,0.329163,0.0,880,0.573,23.0,160.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,479,10,0.486,0.416,0.257,0.822,108,70,2149.0,1650,186,0.355000,195.0,319,0.527,15.6,188.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,louisiana monroe,147,42,0.507,0.428,0.357,0.769,55,80,1938.0,1062,90,0.318000,130.0,449,0.551,20.0,241.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [193]:
id_df = ncaa2_df[["player_id", "name"]]
id_df.rename(columns = {'name':'Player'}, inplace = True) 

In [194]:
id_df.head()

,player_id,Player
0,kenyon-martin-1,Kenyon Martin
1,aj-guyton-1,A.J. Guyton
2,jake-voskuhl-1,Jake Voskuhl
3,khalid-el-amin-1,Khalid El-Amin
4,mike-smith-1,Mike Smith


In [198]:
nba_df = pd.merge(df, id_df, on='Player')
nba_df.tail(20)

,Win Shares,Mins Played (per game),Player,Games,Year,Rank,Points(per game),WS/48,Box +/-,Value Overall Replacement Player,Team,College,Years,Mins Played (Total),Points (Total),Class,player_id
809,7.8,32.4,Trae Young,125,2018,5,22.8,0.093,1.4,3.5,DAL,Oklahoma,2,4046,2844,Above Avg,trae-young-1
810,3.4,15.8,Mohamed Bamba,91,2018,6,5.8,0.112,0.4,0.9,ORL,Texas,2,1440,532,Above Avg,mohamed-bamba-1
811,0.8,31.9,Collin Sexton,131,2018,8,17.8,0.009,-4.4,-2.5,CLE,Alabama,2,4175,2329,Below Avg,collin-sexton-1
812,-0.9,24.9,Kevin Knox,123,2018,9,10.6,-0.014,-5.3,-2.6,NYK,Kentucky,2,3058,1304,Below Avg,kevin-knox-1
813,5.6,27.6,Mikal Bridges,130,2018,10,8.0,0.075,0.3,2.1,PHI,Villanova,2,3593,1042,Above Avg,mikal-bridges-1
814,7.4,29.9,Shai Gilgeous-Alexander,132,2018,11,14.1,0.090,-0.1,1.9,CHO,Kentucky,2,3950,1858,Above Avg,shai-gilgeous-alexander-1
815,4.0,24.5,Miles Bridges,128,2018,12,9.2,0.062,-1.4,0.5,LAC,Michigan State,2,3139,1182,Below Avg,miles-bridges-1
816,0.0,10.8,Jerome Robinson,73,2018,13,3.2,0.002,-4.4,-0.5,LAC,Boston College,2,792,234,Below Avg,jerome-robinson-2
817,0.1,14.7,Zhaire Smith,9,2018,16,4.9,0.025,-4.7,-0.1,PHO,Texas Tech,2,132,44,Below Avg,zhaire-smith-1
818,3.7,19.8,Donte DiVincenzo,70,2018,17,7.4,0.129,1.4,1.2,MIL,Villanova,2,1389,518,Star,donte-divincenzo-1


In [199]:
ncaa2_df.to_csv("NBA_data.csv",index = False, header = True)